<img src="https://unidata.ucar.edu/images/logos/badges/badge_unidata_100.jpg" alt="Unidata Logo" style="float: right; height: 98px;">

# Siphon THREDDS Jupyter Notebook

## Dataset: Best GFS Half Degree Forecast Time Series

Modification: Jurand Bien
___

In [ ]:
# import libraries
import numpy as np
import pandas as pd

from datetime import date, datetime, timedelta
from siphon.catalog import TDSCatalog
from netCDF4 import num2date

import pvlib
from pvlib.location import Location
from pvlib.irradiance import campbell_norman, get_extra_radiation, disc

In [ ]:
# specify location
longitude, latitude, tz, altitude = 19, 51, 'Europe/Warsaw', 270
location = Location(latitude=latitude, longitude=longitude, tz=tz, altitude=altitude)

In [ ]:
# round latitude and longitide to 0.5 degree forecast
def roundhalf(x: float):
    return round(x*2)/2

long = roundhalf(longitude)
lat = roundhalf(latitude)

In [ ]:
# specify forecast period
now = datetime.utcnow()
forecast_days = 7

In [ ]:
# wheather forecast data from THREDDS
best_gfs = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/'
                      'Global_0p5deg/catalog.xml?dataset=grib/NCEP/GFS/Global_0p5deg/Best')
best_ds = best_gfs.datasets[0]
# open remote best GFS
ncss = best_ds.subset()

In [ ]:
# list of interesting fields
gfs_variables = ['Temperature_surface','u-component_of_wind_isobaric','v-component_of_wind_isobaric',
            'Total_cloud_cover_entire_atmosphere_Mixed_intervals_Average','Low_cloud_cover_low_cloud_Mixed_intervals_Average',
            'Medium_cloud_cover_middle_cloud_Mixed_intervals_Average','High_cloud_cover_high_cloud_Mixed_intervals_Average',
            'Total_cloud_cover_boundary_layer_cloud_Mixed_intervals_Average','Total_cloud_cover_convective_cloud']

gfs_columns = ['temp_air','wind_speed_u','wind_speed_v','total_clouds',
               'low_clouds','mid_clouds','high_clouds','boundary_clouds','convect_clouds', 'time']

df = pd.DataFrame()

for gfs_variable in gfs_variables:
    query = ncss.query()
    query.lonlat_point(long, lat).vertical_level(100000).time_range(now, now + timedelta(days=forecast_days))
    query.variables(gfs_variable).accept('netcdf')
    data = ncss.get_data(query)
    gfs_col = np.array(data.variables[gfs_variable][:].data)
    df = pd.concat([df, pd.DataFrame(gfs_col)], axis=1)

time = data.variables['time']
dates = num2date(time[:], units=time.units, calendar='standard', 
                 only_use_cftime_datetimes=False, only_use_python_datetimes=True)

df = pd.concat([df, pd.DataFrame(dates.data)], axis=1)
df.columns = gfs_columns

df.set_index('time', inplace=True)

In [ ]:
def kelvin_to_celsius(temperature):
    """
    Converts Kelvin to celsius.
    Parameters
    ----------
    temperature: numeric
    Returns
    -------
    temperature: numeric
    """
    return temperature - 273.15
    
def uv_to_speed(data):
    """
    Computes wind speed from wind components.
    Parameters
    ----------
    data : DataFrame
    Must contain the columns 'wind_speed_u' and 'wind_speed_v'.
    Returns
    -------
    wind_speed : Series
    """
    wind_speed = np.sqrt(data['wind_speed_u']**2 + data['wind_speed_v']**2)

    return wind_speed

def cloud_cover_to_ghi_linear(cloud_cover, clouds_type, ghi_clear, offset=35):
    """
    Convert cloud cover to GHI using a linear relationship.
    0% cloud cover returns ghi_clear.
    100% cloud cover returns offset*ghi_clear.
    Parameters
    ----------
    cloud_cover: numeric
        Cloud cover in %.
    clouds_type: str
        Different clouds: 'total_clouds', 'low_clouds','mid_clouds','high_clouds','boundary_clouds','convect_clouds'
    ghi_clear: numeric
        GHI under clear sky conditions.
    offset: numeric, default 35
        Determines the minimum GHI.
    Returns
    -------
    ghi: numeric
       Estimated GHI.
    References
    ----------
    Larson et. al. "Day-ahead forecasting of solar power output from
    photovoltaic plants in the American Southwest" Renewable Energy
    91, 11-20 (2016).
    """

    offset = offset / 100.
    cloud_cover = cloud_cover[clouds_type] / 100.
    ghi = (offset + (1 - offset) * (1 - cloud_cover)) * ghi_clear
    return ghi

def cloud_cover_to_irradiance_clearsky_scaling(cloud_cover, clouds_type='total_clouds', method='linear'):
    """
    Estimates irradiance from cloud cover in the following steps:
    1. Determine clear sky GHI using Ineichen model and
       climatological turbidity.
    2. Estimate cloudy sky GHI using a function of
       cloud_cover e.g.
       :py:meth:`~ForecastModel.cloud_cover_to_ghi_linear`
    3. Estimate cloudy sky DNI using the DISC model.
    4. Calculate DHI from DNI and GHI.
    Parameters
    ----------
    cloud_cover : DataFrame
        Cloud cover in %.
    clouds_type: str
        Different clouds: 'total_clouds', 'low_clouds','mid_clouds','high_clouds','boundary_clouds','convect_clouds'        
    method : str, default 'linear'
        Method for converting cloud cover to GHI.
        'linear' is currently the only option.
    Returns
    -------
    irrads : DataFrame
        Estimated GHI, DNI, and DHI.
    """
    solpos = location.get_solarposition(cloud_cover.index)
    cs = location.get_clearsky(cloud_cover.index, model='ineichen', solar_position=solpos)

    method = method.lower()
    if method == 'linear':
        ghi = cloud_cover_to_ghi_linear(cloud_cover, clouds_type, cs['ghi'])
    else:
        raise ValueError('invalid method argument')
    
    dni = disc(ghi, solpos['zenith'], cloud_cover.index)['dni']
    dhi = ghi - dni * np.cos(np.radians(solpos['zenith']))

    irrads = pd.DataFrame({'ghi': ghi, 'dni': dni, 'dhi': dhi}).fillna(0)
    return irrads    

def cloud_cover_to_transmittance_linear(cloud_cover, clouds_type, offset=0.75):
    """
    Convert cloud cover to atmospheric transmittance using a linear
    model.

    0% cloud cover returns offset.
    100% cloud cover returns 0.

    Parameters
    ----------
    cloud_cover : numeric
        Cloud cover in %.
    clouds_type: str
        Different clouds: 'total_clouds', 'low_clouds','mid_clouds','high_clouds','boundary_clouds','convect_clouds'        
    offset : numeric, default 0.75
        Determines the maximum transmittance.
  
    Returns
    -------
    ghi : numeric
        Estimated GHI.
    """
    transmittance = ((100.0 - cloud_cover[clouds_type]) / 100.0) * offset

    return transmittance

def cloud_cover_to_irradiance_campbell_norman(cloud_cover, clouds_type='total_clouds'):
    """
    Estimates irradiance from cloud cover in the following steps:

    1. Determine transmittance using a function of cloud cover e.g.
       :py:meth:`~ForecastModel.cloud_cover_to_transmittance_linear`
    2. Calculate GHI, DNI, DHI using the
       :py:func:`pvlib.irradiance.campbell_norman` model

    Parameters
    ----------
    cloud_cover : DataFrame

    Returns
    -------
    irradiance : DataFrame
    Columns include ghi, dni, dhi
    """
    # in principle, get_solarposition could use the forecast
    # pressure, temp, etc., but the cloud cover forecast is not
    # accurate enough to justify using these minor corrections
    
    solar_position = location.get_solarposition(cloud_cover.index)
    dni_extra = get_extra_radiation(cloud_cover.index)

    transmittance = cloud_cover_to_transmittance_linear(cloud_cover, clouds_type)

    irrads = campbell_norman(solar_position['apparent_zenith'],
                             transmittance, dni_extra=dni_extra)
    irrads = irrads.fillna(0)

    return irrads    
    
def cloud_cover_to_irradiance(cloud_cover, clouds_type='total_clouds', how='clearsky_scaling'):
    """
    Convert cloud cover to irradiance. A wrapper method.
    Parameters
    ----------
    cloud_cover : DataFrame
    clouds_type : Series
    how : str, default 'clearsky_scaling'
          Selects the method for conversion. Can be one of
          clearsky_scaling or campbell_norman.
    Returns
    -------
    irradiance : DataFrame
    Columns include ghi, dni, dhi
    """

    how = how.lower()
    if how == 'clearsky_scaling':
        irrads = cloud_cover_to_irradiance_clearsky_scaling(cloud_cover, clouds_type)
    elif how == 'campbell_norman':
        irrads = cloud_cover_to_irradiance_campbell_norman(cloud_cover, clouds_type)
    else:
        raise ValueError('invalid how argument')

    return irrads

In [ ]:
# convert temperature
df['temp_air'] = kelvin_to_celsius(df['temp_air'])
# covert wind component to wind speed
df['wind_speed'] = uv_to_speed(df)

# resampling to 5min temporal resolution
df_interpol = df.resample('5min').interpolate(method='time')
# cloud types: total_clouds, low_clouds,mid_clouds,high_clouds,boundary_clouds,convect_clouds
resampled_irrads = cloud_cover_to_irradiance(df_interpol, clouds_type='total_clouds', how='clearsky_scaling')

# calculate irradiance estimates from cloud cover
# uses a cloud_cover to ghi model or
# uses a cloud_cover to transmittance to irradiance model
# irrads = cloud_cover_to_irradiance(df, how='clearsky_scaling')

df = df_interpol.join(resampled_irrads, how='outer')

In [ ]:
df = df.loc[:,['ghi','dni','dhi','wind_speed','temp_air']]
df

In [ ]:
resampled_irrads.plot()

In [ ]:
df.to_csv('prognoza_7dni_cloudcover_to_ghi(5min).csv')